Scraping http://www.vaingloryfire.com/vainglory/wiki/heroes to get utilities for hero choices.

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import csv

Go grab the the main wiki page.

In [3]:
page = requests.get('http://www.vaingloryfire.com/vainglory/wiki/heroes')
soup = BeautifulSoup(page.text, 'html.parser')

Go find the name of every hero and the link to their own page.

In [4]:
root = '/vainglory/wiki/heroes'
urls = [link.get('href') for link in soup.find_all('a')]
heroes = {url[len(root) + 1:]:url for url in urls[2:] if url.startswith(root + '/')}
heroes

{u'adagio': u'/vainglory/wiki/heroes/adagio',
 u'ardan': u'/vainglory/wiki/heroes/ardan',
 u'catherine': u'/vainglory/wiki/heroes/catherine',
 u'celeste': u'/vainglory/wiki/heroes/celeste',
 u'fortress': u'/vainglory/wiki/heroes/fortress',
 u'glaive': u'/vainglory/wiki/heroes/glaive',
 u'joule': u'/vainglory/wiki/heroes/joule',
 u'koshka': u'/vainglory/wiki/heroes/koshka',
 u'krul': u'/vainglory/wiki/heroes/krul',
 u'petal': u'/vainglory/wiki/heroes/petal',
 u'ringo': u'/vainglory/wiki/heroes/ringo',
 u'rona': u'/vainglory/wiki/heroes/rona',
 u'saw': u'/vainglory/wiki/heroes/saw',
 u'skaarf': u'/vainglory/wiki/heroes/skaarf',
 u'taka': u'/vainglory/wiki/heroes/taka',
 u'vox': u'/vainglory/wiki/heroes/vox'}

Functions to parse each hero.

In [5]:
def parse_hero(hero, dic=heroes):
    """Parse a hero"""
    page = requests.get('http://www.vaingloryfire.com' + dic[hero])
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup

In [38]:
def get_guides(hero, dic=heroes, lim_on_guides=None):
    """Get all the urls of the guides on the front page"""
    soup = parse_hero(hero, dic=dic)
    urls = [link.get('href') for link in soup.find_all('a')]
    guides = [url for url in urls[2:] if ((url != None) and url.startswith('/vainglory/guide/'))][:lim_on_guides]
    return guides

In [39]:
def clean_threat(threat):
    """Clean a given threat (this is displayed as image files)"""
    results = []
    for s in BeautifulSoup(str(threat)).find_all("img"):
        result = re.search('heroes/(.*).png', str(s))
        results.append(result.group(1))
    return results

def get_threat_to_hero(hero, guide, dic=heroes):
    """Get each threat to a hero, if there are none, go back and find a different guide. 
    This returns a dictionary of threats to lists of heroes."""

    page = requests.get('http://www.vaingloryfire.com' + guide)
    soup = BeautifulSoup(page.text, 'html.parser')
    threats = soup.find_all("div", {"class":"heroes"})
        
    threat_to_hero = {float(threat['data-position']):clean_threat(threat) for threat in threats}
    return threat_to_hero

In [40]:
def get_hero_to_threat(hero, guide, dic=heroes):
    """Swaps the dictionary: for a given hero, 
    gives a dictionary mapping another hero to it's threat"""
    t2h = get_threat_to_hero(hero, guide, dic=dic)
    hero_to_threat = {hero:threat for threat, sublist in t2h.iteritems() for hero in sublist}
    return hero_to_threat

def expected_threat(hero, dic=heroes, lim_on_guides = None):
    """Go through all the guides and get the mean utility for a hero"""
    guides = get_guides(hero, dic=dic, lim_on_guides=lim_on_guides)
    dictionaries = [get_hero_to_threat(hero, guide, dic=dic) for guide in guides]
    hero_to_threat = {hero:sum([d.get(hero, 0) for d in dictionaries])/len(dic) for hero in dic}
    return hero_to_threat
        

Go scrape!

In [41]:
h2t_dictionaries = {hero:expected_threat(hero) for hero in heroes}

Check that we have everything:

In [42]:
set(h2t_dictionaries.keys()) == set(heroes)

True

All herores are a threat to all heroes

In [43]:
[(hero, len(threat)) for hero, threat in h2t_dictionaries.iteritems()]

[(u'adagio', 16),
 (u'koshka', 16),
 (u'ardan', 16),
 (u'joule', 16),
 (u'catherine', 16),
 (u'celeste', 16),
 (u'krul', 16),
 (u'ringo', 16),
 (u'taka', 16),
 (u'saw', 16),
 (u'petal', 16),
 (u'skaarf', 16),
 (u'vox', 16),
 (u'rona', 16),
 (u'fortress', 16),
 (u'glaive', 16)]

Creating the payoff matrices and writing them to file.
If a hero is not considered a threat, it is given a threat level of 5.

In [44]:
A = [[h2t_dictionaries[hero][opponent] for opponent in heroes]for hero in heroes]
B = [[h2t_dictionaries[opponent][hero] for opponent in heroes]for hero in heroes]

In [45]:
for m in ['A', 'B']:
    f = open('%s.csv' %m, 'w')
    csv_wrtr = csv.writer(f)
    for row in eval(m):
        csv_wrtr.writerow(row)
    f.close()

In [46]:
cat A.csv

In [24]:
cat B.csv

In [47]:
heroes.keys()

[u'adagio',
 u'ardan',
 u'vox',
 u'krul',
 u'celeste',
 u'ringo',
 u'catherine',
 u'skaarf',
 u'koshka',
 u'taka',
 u'petal',
 u'rona',
 u'glaive',
 u'saw',
 u'joule',
 u'fortress']

Repeating the above but just with the first guide for each hero.

In [48]:
h2t_dictionaries = {hero:expected_threat(hero, lim_on_guides=1) for hero in heroes}
A = [[h2t_dictionaries[hero][opponent] for opponent in heroes]for hero in heroes]
B = [[h2t_dictionaries[opponent][hero] for opponent in heroes]for hero in heroes]
for m in ['A', 'B']:
    f = open('%s-1.csv' %m, 'w')
    csv_wrtr = csv.writer(f)
    for row in eval(m):
        csv_wrtr.writerow(row)
    f.close()

In [51]:
cat A-1.csv

In [52]:
cat B-1.csv

Repeating the above but with the first *two* guides for each hero.

In [53]:
h2t_dictionaries = {hero:expected_threat(hero, lim_on_guides=2) for hero in heroes}
A = [[h2t_dictionaries[hero][opponent] for opponent in heroes]for hero in heroes]
B = [[h2t_dictionaries[opponent][hero] for opponent in heroes]for hero in heroes]
for m in ['A', 'B']:
    f = open('%s-2.csv' %m, 'w')
    csv_wrtr = csv.writer(f)
    for row in eval(m):
        csv_wrtr.writerow(row)
    f.close()

Repeating the above with the first *three* guides for each hero.

In [54]:
h2t_dictionaries = {hero:expected_threat(hero, lim_on_guides=3) for hero in heroes}
A = [[h2t_dictionaries[hero][opponent] for opponent in heroes]for hero in heroes]
B = [[h2t_dictionaries[opponent][hero] for opponent in heroes]for hero in heroes]
for m in ['A', 'B']:
    f = open('%s-3.csv' %m, 'w')
    csv_wrtr = csv.writer(f)
    for row in eval(m):
        csv_wrtr.writerow(row)
    f.close()

Repeating the above with the first *four* guides for each hero.

In [55]:
h2t_dictionaries = {hero:expected_threat(hero, lim_on_guides=4) for hero in heroes}
A = [[h2t_dictionaries[hero][opponent] for opponent in heroes]for hero in heroes]
B = [[h2t_dictionaries[opponent][hero] for opponent in heroes]for hero in heroes]
for m in ['A', 'B']:
    f = open('%s-4.csv' %m, 'w')
    csv_wrtr = csv.writer(f)
    for row in eval(m):
        csv_wrtr.writerow(row)
    f.close()

Repeating the above with the first *five* guides for each hero.

In [56]:
h2t_dictionaries = {hero:expected_threat(hero, lim_on_guides=5) for hero in heroes}
A = [[h2t_dictionaries[hero][opponent] for opponent in heroes]for hero in heroes]
B = [[h2t_dictionaries[opponent][hero] for opponent in heroes]for hero in heroes]
for m in ['A', 'B']:
    f = open('%s-5.csv' %m, 'w')
    csv_wrtr = csv.writer(f)
    for row in eval(m):
        csv_wrtr.writerow(row)
    f.close()

Repeating the above with the first *six* guides for each hero.

In [57]:
h2t_dictionaries = {hero:expected_threat(hero, lim_on_guides=6) for hero in heroes}
A = [[h2t_dictionaries[hero][opponent] for opponent in heroes]for hero in heroes]
B = [[h2t_dictionaries[opponent][hero] for opponent in heroes]for hero in heroes]
for m in ['A', 'B']:
    f = open('%s-6.csv' %m, 'w')
    csv_wrtr = csv.writer(f)
    for row in eval(m):
        csv_wrtr.writerow(row)
    f.close()

Repeating the above with the first *seven* guides for each hero.

In [61]:
h2t_dictionaries = {hero:expected_threat(hero, lim_on_guides=7) for hero in heroes}
A = [[h2t_dictionaries[hero][opponent] for opponent in heroes]for hero in heroes]
B = [[h2t_dictionaries[opponent][hero] for opponent in heroes]for hero in heroes]
for m in ['A', 'B']:
    f = open('%s-7.csv' %m, 'w')
    csv_wrtr = csv.writer(f)
    for row in eval(m):
        csv_wrtr.writerow(row)
    f.close()